In [1]:
import mysql.connector
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
password = getpass.getpass()

In [2]:
cnx = mysql.connector.connect(user = 'root', password = password,
                              host = '127.0.0.1', database = 'sakila')

In [3]:
cnx.is_connected()

True

In [4]:
cursor = cnx.cursor()

In [5]:
#In this lab, you will be using the Sakila database of movie rentals.

#In order to optimize our inventory, we would like to know which films will be rented next month and we are asked 
#to create a model to predict it.

#Instructions
#Create a query or queries to extract the information you think may be relevant for building the prediction model. 
#It should include some film features and some rental features.
#Read the data into a Pandas dataframe.

In [6]:
query = '''
       select f.film_id, f.title, f.rental_duration, f.rental_rate, r.rental_date, r. return_date, r.rental_id, r.customer_id 
from sakila.inventory i
join film f on i.film_id = f.film_id
join rental r on i.inventory_id = r.inventory_id
        '''


In [7]:
cursor.execute(query)

In [8]:
data = pd.DataFrame(cursor.fetchall())
data.columns = [head[0] for head in cursor.description]

In [9]:
data.head(5)

,film_id,title,rental_duration,rental_rate,rental_date,return_date,rental_id,customer_id
0,1,ACADEMY DINOSAUR,6,0.99,2005-07-08 19:03:15,2005-07-11 21:29:15,4863,431
1,1,ACADEMY DINOSAUR,6,0.99,2005-08-02 20:13:10,2005-08-11 21:35:10,11433,518
2,1,ACADEMY DINOSAUR,6,0.99,2005-08-21 21:27:43,2005-08-30 22:26:43,14714,279
3,1,ACADEMY DINOSAUR,6,0.99,2005-05-30 20:21:07,2005-06-06 00:36:07,972,411
4,1,ACADEMY DINOSAUR,6,0.99,2005-06-17 20:24:00,2005-06-23 17:45:00,2117,170


In [10]:
#Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical 
#variables.

In [11]:
data.dtypes

film_id                     int64
title                      object
rental_duration             int64
rental_rate                object
rental_date        datetime64[ns]
return_date        datetime64[ns]
rental_id                   int64
customer_id                 int64
dtype: object

In [12]:
data.isna().sum()

film_id              0
title                0
rental_duration      0
rental_rate          0
rental_date          0
return_date        183
rental_id            0
customer_id          0
dtype: int64

In [13]:
dataclean = data.dropna()

In [14]:
dataclean.isna().sum()

film_id            0
title              0
rental_duration    0
rental_rate        0
rental_date        0
return_date        0
rental_id          0
customer_id        0
dtype: int64

In [15]:
dataclean['rental_date'] = pd.to_datetime(dataclean['rental_date']).dt.date
dataclean['return_date'] = pd.to_datetime(dataclean['return_date']).dt.date

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\1339741994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['rental_date'] = pd.to_datetime(dataclean['rental_date']).dt.date
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\1339741994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['return_date'] = pd.to_datetime(dataclean['return_date']).dt.date


In [16]:
dataclean['rental_date'] = dataclean['rental_date'].apply(lambda x: x.toordinal())
dataclean['return_date'] = dataclean['return_date'].apply(lambda x: x.toordinal())

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\3505256805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['rental_date'] = dataclean['rental_date'].apply(lambda x: x.toordinal())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\3505256805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['return_date'] = dataclean['return_date'].apply(lambda x: x.toordinal())


In [58]:
dataclean

,film_id,title,rental_duration,rental_rate,rental_date,return_date,rental_id,customer_id,rented_last_month
0,1,ACADEMY DINOSAUR,6,0.99,732135,732138,4863,431,False
1,1,ACADEMY DINOSAUR,6,0.99,732160,732169,11433,518,True
2,1,ACADEMY DINOSAUR,6,0.99,732179,732188,14714,279,True
3,1,ACADEMY DINOSAUR,6,0.99,732096,732103,972,411,False
4,1,ACADEMY DINOSAUR,6,0.99,732114,732120,2117,170,False
...,...,...,...,...,...,...,...,...,...
16039,1000,ZORRO ARK,3,4.99,732095,732097,711,215,False
16040,1000,ZORRO ARK,3,4.99,732112,732114,1493,235,False
16041,1000,ZORRO ARK,3,4.99,732139,732147,6712,226,False
16042,1000,ZORRO ARK,3,4.99,732158,732159,9701,401,False


In [18]:
dataclean.dtypes

film_id             int64
title              object
rental_duration     int64
rental_rate        object
rental_date         int64
return_date         int64
rental_id           int64
customer_id         int64
dtype: object

In [26]:
#Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target 
#variable.

In [27]:
max_rental_date = data['rental_date'].max()
print(max_rental_date)

2006-02-14 15:16:03


In [28]:
print(data['rental_date'].dt.month.unique())
print(data['rental_date'].dt.year.unique())



[7 8 5 6 2]
[2005 2006]


In [29]:
start_date = pd.to_datetime('2005-08-01 00:00:01')
end_date = pd.to_datetime('2006-02-01 23:59:59')

In [30]:
rented_last_month = data[(data['rental_date'].dt.month == 8) & 
                         (data['rental_date'].dt.year == 2005) & 
                         (data['rental_date'] >= start_date) &
                         (data['rental_date'] <= end_date)]

In [31]:
data['rented_last_month'] = data['rental_date'].apply(lambda x: True if start_date <= x <= end_date else False)

In [32]:
data.head()

,film_id,title,rental_duration,rental_rate,rental_date,return_date,rental_id,customer_id,rented_last_month
0,1,ACADEMY DINOSAUR,6,0.99,2005-07-08 19:03:15,2005-07-11 21:29:15,4863,431,False
1,1,ACADEMY DINOSAUR,6,0.99,2005-08-02 20:13:10,2005-08-11 21:35:10,11433,518,True
2,1,ACADEMY DINOSAUR,6,0.99,2005-08-21 21:27:43,2005-08-30 22:26:43,14714,279,True
3,1,ACADEMY DINOSAUR,6,0.99,2005-05-30 20:21:07,2005-06-06 00:36:07,972,411,False
4,1,ACADEMY DINOSAUR,6,0.99,2005-06-17 20:24:00,2005-06-23 17:45:00,2117,170,False


In [33]:
value_counts = data['rented_last_month'].value_counts()

print(value_counts)

False    10358
True      5686
Name: rented_last_month, dtype: int64


In [34]:
#Create a logistic regression model to predict this variable from the cleaned data.
#Evaluate the results.

In [35]:
dataclean = data.dropna()

In [36]:
dataclean.dtypes

film_id                       int64
title                        object
rental_duration               int64
rental_rate                  object
rental_date          datetime64[ns]
return_date          datetime64[ns]
rental_id                     int64
customer_id                   int64
rented_last_month              bool
dtype: object

In [37]:
dataclean['rental_date'] = pd.to_datetime(dataclean['rental_date']).dt.date
dataclean['return_date'] = pd.to_datetime(dataclean['return_date']).dt.date

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\1339741994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['rental_date'] = pd.to_datetime(dataclean['rental_date']).dt.date
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\1339741994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['return_date'] = pd.to_datetime(dataclean['return_date']).dt.date


In [38]:
dataclean['rental_date'] = dataclean['rental_date'].apply(lambda x: x.toordinal())
dataclean['return_date'] = dataclean['return_date'].apply(lambda x: x.toordinal())

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\3505256805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['rental_date'] = dataclean['rental_date'].apply(lambda x: x.toordinal())
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6004\3505256805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataclean['return_date'] = dataclean['return_date'].apply(lambda x: x.toordinal())


In [39]:
cat = dataclean.select_dtypes(include=['object', 'bool'])

cat

,title,rental_rate,rented_last_month
0,ACADEMY DINOSAUR,0.99,False
1,ACADEMY DINOSAUR,0.99,True
2,ACADEMY DINOSAUR,0.99,True
3,ACADEMY DINOSAUR,0.99,False
4,ACADEMY DINOSAUR,0.99,False
...,...,...,...
16039,ZORRO ARK,4.99,False
16040,ZORRO ARK,4.99,False
16041,ZORRO ARK,4.99,False
16042,ZORRO ARK,4.99,False


In [65]:
df_cat = cat.drop('title', axis=1)


In [66]:
categorical = pd.get_dummies(df_cat)
categorical

,rented_last_month,rental_rate_0.99,rental_rate_2.99,rental_rate_4.99
0,False,1,0,0
1,True,1,0,0
2,True,1,0,0
3,False,1,0,0
4,False,1,0,0
...,...,...,...,...
16039,False,0,0,1
16040,False,0,0,1
16041,False,0,0,1
16042,False,0,0,1


In [41]:
numerical_cols = dataclean.select_dtypes(include=['int']).columns
numericals = dataclean[numerical_cols]

In [69]:
final_data = pd.concat([categorical, numericals], axis=1)

In [70]:
final_data

,rented_last_month,rental_rate_0.99,rental_rate_2.99,rental_rate_4.99,film_id,rental_duration,rental_date,return_date,rental_id,customer_id
0,False,1,0,0,1,6,732135,732138,4863,431
1,True,1,0,0,1,6,732160,732169,11433,518
2,True,1,0,0,1,6,732179,732188,14714,279
3,False,1,0,0,1,6,732096,732103,972,411
4,False,1,0,0,1,6,732114,732120,2117,170
...,...,...,...,...,...,...,...,...,...,...
16039,False,0,0,1,1000,3,732095,732097,711,215
16040,False,0,0,1,1000,3,732112,732114,1493,235
16041,False,0,0,1,1000,3,732139,732147,6712,226
16042,False,0,0,1,1000,3,732158,732159,9701,401


In [71]:
X = final_data.drop('rented_last_month', axis=1).values
y = final_data['rented_last_month'].values


In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X,y,test_size = 0.4, random_state=42)

In [73]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X_train, y_train)

In [74]:
predictions = model.predict(X_test)
print(model)

LogisticRegression()


In [75]:
model = LogisticRegression(penalty='l2')


In [76]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [77]:
model.score(X_test,y_test)

1.0

In [78]:
pd.Series(predictions).value_counts()

False    4061
True     2284
dtype: int64

In [79]:
y_pred = model.predict(X_test)

In [80]:
rented_films = y_pred[y_pred == 1]

In [81]:
len(y_pred)

6345

In [82]:
len(final_data)

15861

In [85]:
y_pred_df = pd.DataFrame(y_pred, columns=['prediction'])
result_df = final_data[['film_id']].merge(y_pred_df, left_index=True, right_index=True)
result_df = result_df[['film_id', 'prediction']]
print(result_df)

      film_id  prediction
0           1       False
1           1       False
2           1       False
3           1       False
4           1       False
...       ...         ...
6340      394        True
6341      394        True
6342      394        True
6343      394       False
6344      394       False

[6264 rows x 2 columns]


In [88]:
merged_df = result_df.merge(dataclean[['film_id', 'title']].drop_duplicates(), on='film_id')
print(merged_df[['film_id', 'title', 'prediction']])


      film_id             title  prediction
0           1  ACADEMY DINOSAUR       False
1           1  ACADEMY DINOSAUR       False
2           1  ACADEMY DINOSAUR       False
3           1  ACADEMY DINOSAUR       False
4           1  ACADEMY DINOSAUR       False
...       ...               ...         ...
6259      394     HAMLET WISDOM        True
6260      394     HAMLET WISDOM        True
6261      394     HAMLET WISDOM        True
6262      394     HAMLET WISDOM       False
6263      394     HAMLET WISDOM       False

[6264 rows x 3 columns]


In [89]:
will_be_rented = merged_df[merged_df['prediction'] == True]
print(will_be_rented[['film_id', 'title']])


      film_id             title
5           1  ACADEMY DINOSAUR
7           1  ACADEMY DINOSAUR
8           1  ACADEMY DINOSAUR
9           1  ACADEMY DINOSAUR
11          1  ACADEMY DINOSAUR
...       ...               ...
6255      393    HALLOWEEN NUTS
6256      394     HAMLET WISDOM
6259      394     HAMLET WISDOM
6260      394     HAMLET WISDOM
6261      394     HAMLET WISDOM

[2261 rows x 2 columns]


In [90]:
unique_movies = will_be_rented.drop_duplicates(subset=['title'])
print(unique_movies['title'])

5       ACADEMY DINOSAUR
22        ACE GOLDFINGER
31      ADAPTATION HOLES
40      AFFAIR PREJUDICE
65           AFRICAN EGG
              ...       
6197         GUYS FALCON
6206       HALF OUTFIELD
6232        HALL CASSIDY
6247      HALLOWEEN NUTS
6256       HAMLET WISDOM
Name: title, Length: 369, dtype: object


In [91]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0
